In [1]:
# Import and data read

import numpy as np
import tensorflow as tf
import nltk
from collections import Counter
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer


with open('reviews.txt', 'r') as f:
  reviews = f.read()
with open('labels.txt', 'r') as f:
  labels = f.read()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
reviews = "".join([char for char in reviews if char not in string.punctuation])

# all_words = nltk.word_tokenize(reviews)
# all_words = [word for word in all_words if word.lower() not in stop_words]


reviews = reviews.split('\n')
labels = labels.split('\n')

In [3]:
import itertools

reviews_tokenized = []
for review in reviews:
  splitted_review = nltk.word_tokenize(review)
  splitted_review = [PorterStemmer().stem(w).strip() for w in splitted_review]
  splitted_review = [WordNetLemmatizer().lemmatize(w) for w in splitted_review]
  reviews_tokenized.append(splitted_review)
  
reviews_unrolled = list(itertools.chain(*reviews_tokenized))
  
labels = [1 if label == "positive" else 0 for label in labels]

In [4]:
empty_idx = []
for i, review in enumerate(reviews_tokenized):
  if len(review) == 0:
    empty_idx.append(i)
    
for i in empty_idx:
  reviews_tokenized.pop(i)
  labels.pop(i)

In [5]:
empty_idx

[25000]

In [6]:
vocab_size = 70000

word_counter = Counter(reviews_unrolled)
# word_counter = dict(word_counter.most_common(vocab_size))
word2index = {k:i for i,k in enumerate(word_counter.keys(), start = 3)}

reviews_int = []
for review in reviews_tokenized:
  cur_review = [1]
  for word in review:
    if word in word2index.keys():
      cur_review.append(word2index[word])
    else:
      cur_review.append(2)
  reviews_int.append(cur_review)

In [7]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded_reviews = pad_sequences(reviews_int, maxlen = 500, padding = 'post', truncating = 'post')

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(padded_reviews, labels, test_size = 0.2, random_state = 1)

# X_train = X_train.reshape(20000, 200, 1)

y_train = np.array(y_train).reshape(20000, 1)
y_test = np.array(y_test).reshape(5000, 1)

In [9]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import GlobalMaxPool1D, BatchNormalization, Dense, RNN, GRU, LSTM, TimeDistributed, Bidirectional, Activation, Embedding, Input, Conv1D, Dropout
import tensorflow as tf
import keras.backend as K

dropout_rate = 0.5

inputs = Input(shape = (X_train.shape[1:]))
mask = tf.keras.layers.Lambda(lambda inputs: K.not_equal(inputs, 0))(inputs)
x = Embedding(input_dim = vocab_size, output_dim = 128, input_length = 200)(inputs)
x = Conv1D(filters = 200, kernel_size = 13, strides = 1, padding = 'same', activation = 'relu')(x)
x = BatchNormalization()(x)
x = Dropout(dropout_rate)(x)
x = GRU(128, return_sequences = True)(x)
x = BatchNormalization()(x)
x = Dropout(dropout_rate)(x)
x = GRU(128, return_sequences = False)(x)
x = BatchNormalization()(x)
x = Dropout(dropout_rate)(x)
x = Dense(512, activation = 'relu')(x)
x = Dropout(dropout_rate)(x)
outputs = Dense(1, activation = 'sigmoid')(x)

model = Model(inputs = inputs, outputs = outputs)

model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), loss = 'binary_crossentropy', metrics = ['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 500)]             0         
                                                                 
 embedding (Embedding)       (None, 500, 128)          8960000   
                                                                 
 conv1d (Conv1D)             (None, 500, 200)          333000    
                                                                 
 batch_normalization (BatchN  (None, 500, 200)         800       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 500, 200)          0         
                                                                 
 gru (GRU)                   (None, 500, 128)          126720    
                                                             

In [10]:
history = model.fit(X_train, y_train, epochs = 100, batch_size = 128, validation_data = (X_test, y_test))

Epoch 1/100
157/157 [==============================] - 25s 137ms/step - loss: 0.7807 - accuracy: 0.4963 - val_loss: 0.6937 - val_accuracy: 0.5080
Epoch 2/100
157/157 [==============================] - 21s 132ms/step - loss: 0.7271 - accuracy: 0.5076 - val_loss: 0.7124 - val_accuracy: 0.5084
Epoch 3/100
157/157 [==============================] - 21s 131ms/step - loss: 0.7111 - accuracy: 0.5010 - val_loss: 0.6936 - val_accuracy: 0.5080
Epoch 4/100
157/157 [==============================] - 21s 131ms/step - loss: 0.7029 - accuracy: 0.4928 - val_loss: 0.6956 - val_accuracy: 0.4918
Epoch 5/100
157/157 [==============================] - 21s 131ms/step - loss: 0.6978 - accuracy: 0.5074 - val_loss: 0.6947 - val_accuracy: 0.4916
Epoch 6/100
157/157 [==============================] - 21s 132ms/step - loss: 0.6964 - accuracy: 0.5046 - val_loss: 0.6936 - val_accuracy: 0.5102
Epoch 7/100
157/157 [==============================] - 21s 131ms/step - loss: 0.6873 - accuracy: 0.5209 - val_loss: 0.6968 -

KeyboardInterrupt: 

In [ ]:
vocab_size